In [1]:
# 读取数据 用电影的id来表示电影 便于计算

def load_movielens(path='ml-100k'):
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = id

    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs
prefs= load_movielens()

In [2]:
# 划分数据集 按照7：1 划分数据集
import random
import numpy
def split_data(data, M, k, seed):
    
    test = []
    train = []
    random.seed(seed)
    for user in data:
        if random.randint(0, M) == k:
            test.append([user, data[user]])
        else:
            train.append([user, data[user]])
    return train, test
train,test = split_data(prefs,8,5,1)
def changenp(data):     # 把每个用户的评分放到一个大矩阵中 
    a = []
    b = []
    for i in range(len(data)):
        for j in range(1682):
            a.append(data[i][1].get(str(j+1),0.0))
        b.append(a)
        a = []
    b = numpy.array(b)
    return b

a = changenp(train)
b = changenp(test)
b

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
import numpy as np


def sgd(data_matrix, k, alpha, lam, max_cycles):
    """使用梯度下降法进行矩阵分解。

    Args:
    - data_matrix: mat, 用户物品矩阵
    - k: int, 分解矩阵的参数
    - alpha: float, 学习率
    - lam: float, 正则化参数
    - max_cycles: int, 最大迭代次数

    Returns:
    p,q: mat, 分解后的矩阵
    """
    m, n = np.shape(data_matrix)
    # initiate p & q
    p = np.mat(np.random.random((m, k)))
    q = np.mat(np.random.random((k, n)))

    # start training
    for step in range(max_cycles):
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = data_matrix[i, j]
                    for r in range(k):
                        error = error - p[i, r] * q[r, j]
                    for r in range(k):
                        p[i, r] = p[i, r] + alpha * (2 * error * q[r, j] - lam * p[i, r])
                        q[r, j] = q[r, j] + alpha * (2 * error * p[i, r] - lam * q[r, j])

        loss = 0.0
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = 0.0
                    for r in range(k):
                        error = error + p[i, r] * q[r, j]
                    # calculate loss function
                    loss = (data_matrix[i, j] - error) * (data_matrix[i, j] - error)
                    for r in range(k):
                        loss = loss + lam * (p[i, r] * p[i, r] + q[r, j] * q[r, j]) / 2

        if loss < 0.001:
            break
        if step % 1 == 0:
            print("\titer: %d, loss: %f" % (step, loss))
    print(p)
    return p, q
p,q=sgd(a,2,0.001,0.01,10)

	iter: 0, loss: 5.690922
	iter: 1, loss: 4.684306
	iter: 2, loss: 3.761012
	iter: 3, loss: 2.964148
	iter: 4, loss: 2.300761
	iter: 5, loss: 1.761959
	iter: 6, loss: 1.332377
	iter: 7, loss: 0.994921
	iter: 8, loss: 0.733201
	iter: 9, loss: 0.532649
[[1.31813436 1.18582104]
 [0.99224822 1.46253314]
 [0.89803397 1.52026401]
 ...
 [0.97583735 0.74569628]
 [1.016599   1.30621706]
 [1.0185386  1.63884834]]


In [7]:
def prediction(data_matrix, p, q, user):
    """为用户未互动的项打分

    Args:
    - data_matrix: mat, 原始用户物品矩阵
    - p: mat, 分解后的矩阵p
    - q: mat, 分解后的矩阵q
    - user: int, 用户的id

    Returns:
    - predict: list, 推荐列表
    """
    n = np.shape(data_matrix)[1]
    predict = {}
    for j in range(n):
        if data_matrix[user, j] == 0:
            predict[j] = (p[user,] * q[:, j])[0, 0]

    # 按照打分从大到小排序
    return sorted(predict.items(), key=lambda d: d[1], reverse=True)
predict = prediction(a,p,q,6)

In [8]:


def top_k(predict, n):
    """为用户推荐前 n 个物品

    Args:
    - predict: list, 排好序的物品列表
    - k: int, 推荐的物品个数

    :return: top_recom, list, top n 个物品
    """
    top_recom = []
    len_result = len(predict)
    if n >= len_result:
        top_recom = predict
    else:
        for i in range(n):
            top_recom.append(predict[i])
    return top_recom
top = top_k(predict,5)
top



[(407, 5.394292610595411),
 (168, 5.340629691496618),
 (113, 5.183884409088389),
 (301, 5.155391176655166),
 (473, 5.150107635309131)]

In [11]:
# 召回率
def recall(train, test, N,predict):
    hit = 0
    all = 0
    for user in range(len(test)):
        tu = test[user]
        rank = top_k(predict, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)
recall(a,b,1000,predict)

0.0028071637839096803

In [13]:


# 准确度
def precision(train, test, N, predict):
    hit = 0
    all = 0
    for user in range(len(test)):
        tu = test[user]
        rank = top_k(predict, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += N
    return hit / (all * 1.0)
precision(a,b,1000,predict)



0.004721649484536083

In [14]:
# 覆盖率
def coverage(train,N, predict):
    recommend_items = set()
    all_items = set()
    for user in range(len(train)):
        for item in train[user]:
            if item != 0:
                all_items.add(user)
        rank = top_k(predict, N)
        for item, pui in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)
coverage(a,5,predict)

0.00591016548463357

In [16]:
import math

# 新颖度
def popularity(train, N, predict):
    item_popularity = dict()
    for user in range(len(train)):
        i = 0
        for item in train[user]:
            i = i+1
            if item != 0:
                item_popularity[str(i)]=item_popularity.get(str(i),0)
    ret = 0
    n = 0
    for user in range(len(train)):
        rank = top_k(predict, N)
        for item, pui in rank:
            ret += math.log(1 + item_popularity.get(item,0.1))
            n += 1
    ret /= n * 1.0
    return ret
popularity(a, 5, predict)



0.09531017980431797